In [1]:
import numpy as np 
import pandas as pd
import sys

dataX = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/train_image.csv"#sys.argv[1]
dataY = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/train_label.csv"#sys.argv[2]

TEST = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/test_image.csv"#sys.argv[3]

def load_data(X,Y, T):

    train_data = pd.read_csv(X, header=None)
    train_labels = pd.read_csv(Y, header=None)
    test_data = pd.read_csv(T, header=None)
    
    
    
    return train_data, train_labels, test_data

first , second, third = load_data(dataX, dataY, TEST)
print(first.shape , second.shape, third.shape)





(60000, 784) (60000, 1) (10000, 784)


In [2]:
def sigmoid(X):
    z =  1/(1+np.exp(-X))
    return z

In [3]:

def softmax(X):
    e = np.exp(X)
    e = e/e.sum()
    
    return e

def sigmoid_back(A,Z):
    s = sigmoid(Z)
    return A*s*(1-s)

In [4]:
def shuffle_data(X,Y):
    #merge = np.concatenate((X,Y), axis= 1)
    X,Y = np.array(X), np.array(Y)
    #np.random.shuffle(merge)
    #print(X.shape, Y.shape)
    #X,Y = merge[:,:-1], merge[:, -1]
    #Y = np.reshape(Y, (len(Y), 1))
    #print(Y[59999][0])
    Yonehot = np.zeros((X.shape[0], 10))
    for i in range(Y.shape[0]):
        Yonehot[i][Y[i][0]] = 1
        
    merge = np.concatenate((X, Yonehot), axis=1)
    np.random.shuffle(merge)
    a, b =  merge[:,:X.shape[1]], merge[:, X.shape[1]:]
    #a = np.reshape(a, (X.shape[0], X.shape[1]-1))
    #b = np.reshape(b, (X.shape[0], 10))
    
    return a, b

In [5]:
batch_size = 32
learning_rate = 0.001
epochs = 50

params = {
    'w1':np.random.randn(512, 784)*np.sqrt(1.0 / 784),
    'b1':np.random.randn(512,1)*np.sqrt(1.0 / 784),
    'w2':np.random.randn(256, 512)*np.sqrt(1.0 /512),
    'b2':np.random.randn(256,1)*np.sqrt(1.0 / 512),
    'w3':np.random.randn(10,256)*np.sqrt(1.0 / 256),
    'b3':np.random.randn(10,1)*np.sqrt(1.0 / 256)
}


def forward_pass(params, X):
    #print(X.shape)
    fpassdict = {}
    
    fpassdict['a1'] = np.dot(params['w1'], X.T) + params['b1']
    fpassdict['z1'] = sigmoid(fpassdict['a1'])
    
    #print(fpassdict['z1'].shape)
    fpassdict['a2'] = np.dot(params['w2'], fpassdict['z1']) + params['b2']
    fpassdict['z2'] = sigmoid(fpassdict['a2'])
    
    #print(fpassdict['z2'].shape)
    fpassdict['a3'] = np.dot(params['w3'], fpassdict['z2']) + params['b3']
    fpassdict['z3'] = softmax(fpassdict['a3'])
    
    #print(fpassdict['z3'].shape)
    
    return fpassdict


def create_batches(X,Y, batch_size=32):
    mini_batches = list()
    
    length_of_batch = X.shape[0]//batch_size
    
    #print(length_of_batch)
    
    for i in range(batch_size):
        
        b = X[i*length_of_batch:(i+1)*length_of_batch, :]
        
        y = Y[i*length_of_batch:(i+1)*length_of_batch, :]
        
        
        mini_batches.append((b,y))
    
    
        
    if X.shape[0]%length_of_batch != 0:
        
        #print("lastone")
        b = X[batch_size*(X.shape[0]//batch_size):, :]
        
        y = Y[batch_size*(X.shape[0]//batch_size):, :]
        
       
        
        mini_batches.append((b,y))
        
    return mini_batches
         
    
    
    
    

In [6]:
def backward_pass(X,Y, para, og):
    print("in back pass")
    n = Y.shape[1]
    Y = Y.T
    print(para.keys(), og.keys())
    dz3 = para['z3'] - Y
    
    dw3 = np.dot(dz3, para['z2'].T )/n
    db3 = np.sum(dz3, axis=1, keepdims=True)/n
    
    da2 = np.dot(og['w3'].T, dz3)
    dz2 = sigmoid_back(da2, para['a2'])
    
    dw2 = np.dot(dz2,para['z1'].T)/n
    db2 = np.sum(dz2, axis=1, keepdims=True)/n

    da1 = np.dot(og['w2'].T, dz2)
    dz1 = sigmoid_back(da1, para['a1'])
    
    dw1 = np.dot(dz1, X)/n
    db1 = np.sum(dz1, axis=1, keepdims=True)
    
    gradient = {
        'dw3':dw3,
        'db3':db3,
        'dw2':dw2,
        'db2':db2,
        'dw1':dw1,
        'db1':db1
    }
    
    return gradient

In [7]:
def train_model(X, Y, parameters, batch_size = 1, epochs = 1 , learning_rate = 0.001):
    print(parameters.keys())
    for i in range(epochs):
        dataX, dataY = shuffle_data(X,Y)
        bitches = create_batches(dataX, dataY, batch_size)
        #print(len(bitches))
        for b in bitches:
            xtrain, ytrain = b
            print(xtrain.shape, ytrain.shape)
            fpass = forward_pass(parameters, xtrain)
            bpass = backward_pass(xtrain, ytrain, fpass, parameters)
            print("parameters: ", parameters['w1'])
            parameters['w1'] = parameters['w1']-(learning_rate*bpass['dw1'])
            parameters['b1'] = parameters['b1']-(learning_rate*bpass['db1'])
            parameters['w2'] = parameters['w2']-(learning_rate*bpass['dw2'])
            parameters['b2'] = parameters['b2']-(learning_rate*bpass['db2'])
            parameters['w3'] = parameters['w3']-(learning_rate*bpass['dw3'])
            parameters['b3'] = parameters['b3']-(learning_rate*bpass['db3'])
            
    return parameters

In [8]:
final_weights = train_model(first, second, params)

dict_keys(['w1', 'b1', 'w2', 'b2', 'w3', 'b3'])
(60000, 784) (60000, 10)
in back pass
dict_keys(['a1', 'z1', 'a2', 'z2', 'a3', 'z3']) dict_keys(['w1', 'b1', 'w2', 'b2', 'w3', 'b3'])
parameters:  [[ 0.0017967   0.01063578  0.00810841 ...  0.06543299  0.01711715
   0.0381452 ]
 [ 0.04308385 -0.00575438 -0.03413063 ...  0.00111338 -0.04160116
  -0.02618649]
 [-0.02204323  0.02787152 -0.01640722 ...  0.06608088 -0.00318517
   0.05583309]
 ...
 [-0.01267153  0.01414039  0.009985   ...  0.00093593  0.03749905
  -0.02023224]
 [-0.06878733  0.03931028 -0.03387239 ... -0.02973104  0.0492104
   0.02447391]
 [-0.06882624 -0.02314565  0.00896002 ...  0.0373804   0.01357169
  -0.02804321]]


In [10]:
type(final_weights)

dict

In [61]:
import numpy as np 
import pandas as pd
import sys

dataX = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/train_image.csv"#sys.argv[1]
dataY = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/train_label.csv"#sys.argv[2]

TEST = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/test_image.csv"#sys.argv[3]

def load_data(X,Y, T):

    train_data = pd.read_csv(X, header=None)
    train_labels = pd.read_csv(Y, header=None)
    test_data = pd.read_csv(T, header=None)
    
    
    
    return train_data, train_labels, test_data


def sigmoid(X):
    z =  1/(1+np.exp(-X))
    return z

def softmax(X):
    e = np.exp(X)
    e = e/e.sum()
    
    return e

def sigmoid_back(A,Z):
    s = sigmoid(Z)
    return A*s*(1-s)


def create_batches(X,Y, batch_size=32):
    mini_batches = list()
    
    length_of_batch = X.shape[0]//batch_size
    
    #print(length_of_batch)
    
    for i in range(batch_size):
        
        b = X[i*length_of_batch:(i+1)*length_of_batch, :]
        
        y = Y[i*length_of_batch:(i+1)*length_of_batch, :]
        
        
        mini_batches.append((b,y))
    
    
        
    if X.shape[0]%length_of_batch != 0:
        
        #print("lastone")
        b = X[batch_size*(X.shape[0]//batch_size):, :]
        
        y = Y[batch_size*(X.shape[0]//batch_size):, :]
        
       
        
        mini_batches.append((b,y))
        
    return mini_batches


def forward_pass(params, X)->dict:
    #print(X.shape)
    fpassdict = {}
    
    fpassdict['z1'] = np.dot(params['w1'], X.T) + params['b1']
    fpassdict['a1'] = sigmoid(fpassdict['z1'])
    
    #print(fpassdict['z1'].shape)
    fpassdict['z2'] = np.dot(params['w2'], fpassdict['a1']) + params['b2']
    fpassdict['a2'] = sigmoid(fpassdict['z2'])
    
    #print(fpassdict['z2'].shape)
    fpassdict['z3'] = np.dot(params['w3'], fpassdict['a2']) + params['b3']
    fpassdict['a3'] = softmax(fpassdict['z3'])
    
    #print(fpassdict['z3'].shape)
    
    return fpassdict

def backward_pass(X,Y, para, og)->dict:
    #print("in back pass")
    n = Y.shape[1]
    Y = Y.T
    #print(para.keys(), og.keys())
    dz3 = para['a3'] - Y
    
    dw3 = np.dot(dz3, para['a2'].T )/n
    db3 = np.sum(dz3, axis=1, keepdims=True)/n
    
    da2 = np.dot(og['w3'].T, dz3)
    dz2 = sigmoid_back(da2, para['z2'])
    
    dw2 = np.dot(dz2,para['a1'].T)/n
    db2 = np.sum(dz2, axis=1, keepdims=True)/n

    da1 = np.dot(og['w2'].T, dz2)
    dz1 = sigmoid_back(da1, para['a1'])
    
    dw1 = np.dot(dz1, X)/n
    db1 = np.sum(dz1, axis=1, keepdims=True)/n
    
    gradient = {
        'dw3':dw3,
        'db3':db3,
        'dw2':dw2,
        'db2':db2,
        'dw1':dw1,
        'db1':db1
    }
    
    return gradient

def train_model(X, Y, parameters, batch_size = 32, epochs = 10 , learning_rate = 0.0001):
    #print(parameters.keys())
    for i in range(epochs):
        print("Starting epoch: ", i)
        dataX, dataY = shuffle_data(X,Y)
        bitches = create_batches(dataX, dataY, batch_size)
        #print(len(bitches))
        print("completed processing for batch: ", end = " ")
        j = 1
        for b in bitches:
            print("#", end=" ")
            j+=1
            xtrain, ytrain = b
            #print(xtrain.shape, ytrain.shape)
            fpass = forward_pass(parameters, xtrain)
            bpass = backward_pass(xtrain, ytrain, fpass, parameters)
            #print("parameters: ", parameters['w1'])
            parameters['w1'] = parameters['w1']-(learning_rate*bpass['dw1'])
            parameters['b1'] = parameters['b1']-(learning_rate*bpass['db1'])
            parameters['w2'] = parameters['w2']-(learning_rate*bpass['dw2'])
            parameters['b2'] = parameters['b2']-(learning_rate*bpass['db2'])
            parameters['w3'] = parameters['w3']-(learning_rate*bpass['dw3'])
            parameters['b3'] = parameters['b3']-(learning_rate*bpass['db3'])
        print("\n")
        for k in parameters.keys():
            print(k, np.amin(parameters[k]), np.amax(parameters[k]))
    
    
    return parameters





batch_size = 32
learning_rate = 0.001
epochs = 50

params = {
    'w1':np.random.rand(512, 784)*np.sqrt(1.0 / 784),
    'b1':np.random.rand(512,1)*np.sqrt(1.0 / 784),
    'w2':np.random.rand(256, 512)*np.sqrt(1.0 /512),
    'b2':np.random.rand(256,1)*np.sqrt(1.0 / 512),
    'w3':np.random.rand(10,256)*np.sqrt(1.0 / 256),
    'b3':np.random.rand(10,1)*np.sqrt(1.0 / 256)
}


first , second, third = load_data(dataX, dataY, TEST)

final_weights = train_model(first, second, params)


Starting epoch:  0
completed processing for batch:  # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

w1 1.3753552220017922e-07 0.03577621906442731
b1 1.311721342130682e-05 0.03569250225494073
w2 4.0045319678227004e-07 0.044194469955980586
b2 1.7500107968774124e-05 0.04392265750721743
w3 0.05430916751700282 0.1294082333581914
b3 0.061389088027711015 0.12107079567710477
Starting epoch:  1
completed processing for batch:  # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

w1 2.1397304078324982e-07 0.03590147307485403
b1 1.4155999770769804e-05 0.035693467629270656
w2 9.69865055558865e-07 0.044195365068326026
b2 1.8201531633764373e-05 0.04392352372004109
w3 0.10851696642982314 0.19663374161972436
b3 0.11980459512104866 0.1806402764698964
Starting epoch:  2
completed processing for batch:  # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

w1 3.0629523724415414e-07 0.03610441460492623
b1 1.5713934392145245e-05 0.03569491519048566
w2 1.8215343747

In [62]:
print(final_weights.keys())

dict_keys(['w1', 'b1', 'w2', 'b2', 'w3', 'b3'])


In [63]:
for k in final_weights.keys():
    print(k, np.amin(final_weights[k]), np.amax(final_weights[k]))

w1 3.200572271021075e-07 0.03947970640343245
b1 4.1024905244557996e-05 0.03571842966818085
w2 1.564002087319338e-05 0.044221988753738076
b2 3.657153709492055e-05 0.04394588380691333
w3 0.5421933527343338 0.7334858245772525
b3 0.5857768667361716 0.6929886159993253


In [64]:
out = forward_pass(final_weights, first)

In [66]:
out['z3']

array([[159.91181839, 159.91181839, 159.91181839, ..., 159.91181839,
        159.91181839, 159.91181839],
       [180.61112906, 180.61112906, 180.61112906, ..., 180.61112906,
        180.61112906, 180.61112906],
       [160.98365237, 160.98365237, 160.98365237, ..., 160.98365237,
        160.98365237, 160.98365237],
       ...,
       [169.5531151 , 169.5531151 , 169.5531151 , ..., 169.5531151 ,
        169.5531151 , 169.5531151 ],
       [158.73263678, 158.73263678, 158.73263678, ..., 158.73263678,
        158.73263678, 158.73263678],
       [160.89190699, 160.89190699, 160.89190699, ..., 160.89190699,
        160.89190699, 160.89190699]])